In [7]:
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime
import sys
sys.path.append('../')
from user_packages import hashing

In [8]:
# db connect
conn = sqlite3.connect('../full_metadata.db')

In [1]:
query = """
SELECT distinct
	PhysicalAttributeHashKey AS OLD_HASH
	, SERVER_NAME
  , "DATABASE_NAME"
  , SCHEMA_NAME
  , TABLE_NAME
  , [Logical Name] AS COLUMN_NAME
FROM
	rv_s_PhysicalAttribute_XRMMetadata
"""
df = pd.read_sql_query(query, conn)
#df

NameError: name 'pd' is not defined

In [2]:
# derive keyphrase columns
df['NEW_KEYPHRASE'] = df.apply(
  lambda row: 
    row['SERVER_NAME']
    + '.' + row['DATABASE_NAME']
    + '.' + row['SCHEMA_NAME']
    + '.' + row['TABLE_NAME']
    + '.' + row['COLUMN_NAME']
  , axis=1
)

df['PhysicalStructureKeyPhrase'] = df.apply(
  lambda row: 
    row['SERVER_NAME']
    + '.' + row['DATABASE_NAME']
    + '.' + row['SCHEMA_NAME']
    + '.' + row['TABLE_NAME']
  , axis=1
)
df['PhysicalStructurePhysicalAttributeKeyPhrase'] = df.apply(  lambda row: row['PhysicalStructureKeyPhrase'] + ':' + row['NEW_KEYPHRASE'], axis=1)


# hash the keyphrases
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'NEW_HASH'
  , columns = ['NEW_KEYPHRASE']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'PhysicalStructureHashKey'
  , columns = ['PhysicalStructureKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'PhysicalStructurePhysicalAttributeHashKey'
  , columns = ['PhysicalStructurePhysicalAttributeKeyPhrase']
)

#df

NameError: name 'df' is not defined

In [11]:
def fix_hash(OLD_HASH, NEW_KEYPHRASE, NEW_HASH, PhysicalStructureHashKey, PhysicalStructurePhysicalAttributeHashKey):
  #print("I'm iterating")
  sql = """
    UPDATE rv_h_PhysicalAttribute
    SET
      PhysicalAttributeHashKey = ?
      , PhysicalAttributeKeyPhrase = ?
    WHERE
      PhysicalAttributeHashKey = ?
  """
  conn.execute(sql, (NEW_HASH, NEW_KEYPHRASE, OLD_HASH))
  conn.commit()

  sql = """
    UPDATE rv_s_PhysicalAttribute_XRMMetadata
    SET
      PhysicalAttributeHashKey = ?
    WHERE
      PhysicalAttributeHashKey = ?
  """
  conn.execute(sql, (NEW_HASH, OLD_HASH))
  conn.commit()

  sql = """
    UPDATE rv_s_PhysicalAttribute_AnalysisTag
    SET
      PhysicalAttributeHashKey = ?
    WHERE
      PhysicalAttributeHashKey = ?
  """
  conn.execute(sql, (NEW_HASH, OLD_HASH))
  conn.commit()

  sql = """
    UPDATE rv_l_PhysicalStructurePhysicalAttribute
    SET
      PhysicalAttributeHashKey = ?
      , PhysicalStructureHashKey = ?
      , PhysicalStructurePhysicalAttributeHashKey = ?
    WHERE
      PhysicalAttributeHashKey = ?
  """
  conn.execute(sql, (NEW_HASH, PhysicalStructureHashKey, PhysicalStructurePhysicalAttributeHashKey, OLD_HASH))
  conn.commit()

In [12]:
result = [
  fix_hash(
    row[0]
    , row[1]
    , row[2]
    , row[3]
    , row[4]
  ) for row in zip(
    df['OLD_HASH']
    , df['NEW_KEYPHRASE']
    , df['NEW_HASH']
    , df['PhysicalStructureHashKey']
    , df['PhysicalStructurePhysicalAttributeHashKey']
  )
]
